In [46]:
# 0) Prepare data
# 1) Design model (input, output size, forward pass)
# 2) Construct loss and optimizer
# 3) training loop
#       - Forward pass: compute prediction
#       - Backward pass: gradients
#       - update weights

In [47]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [48]:
# 0) prepare data!
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1234)

#scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32)).reshape(-1,1)
y_test = torch.from_numpy(y_test.astype(np.float32)).reshape(-1,1)


n_samples, n_features = X.shape

In [49]:
# 1) model
input_size = n_features
output_size = 1

class LogtisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogtisticRegression,self).__init__()
        self.linear = nn.Linear(n_input_features,1)

    def forward(self,x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogtisticRegression(n_features)



In [50]:
# 2) loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [51]:
# 3) training loop
num_epochs = 1000
for epoch in range(num_epochs):
    #forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted,y_train)

    #backwards pass
    loss.backward()

    #update
    optimizer.step()
    optimizer.zero_grad()

    if (epoch+1) % 100 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 1000, loss = 0.0885
epoch: 2000, loss = 0.0692
epoch: 3000, loss = 0.0605
epoch: 4000, loss = 0.0551
epoch: 5000, loss = 0.0513
epoch: 6000, loss = 0.0484
epoch: 7000, loss = 0.0461
epoch: 8000, loss = 0.0442
epoch: 9000, loss = 0.0426
epoch: 10000, loss = 0.0412


In [52]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9561
